In [2]:
def preprocess_image(image_path):
  hr_image = cv2.imread(image_path)
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.png" % filename)
  print("Saved as %s.png" % filename)

In [3]:
import math
def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return 10.0 * (1.0 / math.log(10)) * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - 
y_true))))

In [4]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
# Load the pre-trained ESRGAN model
module = hub.load(SAVED_MODEL_PATH)

class ESRGANModel(tf.keras.layers.Layer):
    def __init__(self, module, **kwargs):
        super().__init__(**kwargs)
        self.module = module

    def call(self, inputs):
        return self.module(inputs)

inputs = tf.keras.Input(shape=(None, None, 3))
esrgan = ESRGANModel(module)
outputs = esrgan(inputs)

# Create the model instance
model = tf.keras.Model(inputs, outputs)

# Freeze all layers except the last layer
for layer in model.layers[:-1]:
    layer.trainable = False

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),
              loss=tf.keras.losses.MeanSquaredError(),metrics=[PSNR])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
print(module.signatures)

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(input_0) at 0x7F2DB832DA00>})


In [6]:
from PIL import Image
def downscale_image(image):
  image_size = []
  if len(image.shape) == 3:
    image_size = [image.shape[1], image.shape[0]]
  else:
    raise ValueError("Dimension mismatch. Can work only on single image.")

  image = tf.squeeze(
      tf.cast(
          tf.clip_by_value(image, 0, 255), tf.uint8))

  lr_image = np.asarray(
    Image.fromarray(image.numpy())
    .resize([image_size[0] // 4, image_size[1] // 4],
              Image.BICUBIC))

  lr_image = tf.cast(lr_image, tf.float32)
  return lr_image

In [20]:
model = tf.keras.Sequential([
    hub.KerasLayer(SAVED_MODEL_PATH, input_shape=(None, None, 3))
])

In [21]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, None, None, 3)     4605955   
                                                                 
Total params: 4,605,955
Trainable params: 0
Non-trainable params: 4,605,955
_________________________________________________________________


In [26]:
img = cv2.imread("/content/ch2_tmc_ndn_20191202T0639511671_b_bdt_d18.png")
print(img.shape)
img = tf.convert_to_tensor(img)
img = tf.expand_dims(img,0)
print(img.shape)

(8068, 214, 3)
(1, 8068, 214, 3)


In [ ]:
new_img = model.predict(img)

In [ ]:
import os
import numpy as np
image_folder = "./test"
image_paths = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')]
itr = 0
for image_path in image_paths:
    itr =  itr + 1
    image = cv2.imread(image_path)
    print(image.shape)
    downscaled = downscale_image(image)
    print(downscaled.shape)
    save_image(downscaled,f'train/downscaled{itr}')